# Create a simple ISA descriptor

This example creates minimal metadata for a single study ISA descriptor with no assay declared. 

It shows how to serialize (write) the ISA Model content to ISA-Tab and ISA-JSON formats.

In [1]:
from isatools.model import *
from isatools.create.model import *
import datetime

## Study metadata

In [2]:
investigation = Investigation()
study = Study(filename="s_study.txt")
study.identifier = "S1"
study.title = "My Simple ISA Study"
study.description = "We could alternatively use the class constructor's parameters to set some default " \
          "values at the time of creation, however we want to demonstrate how to use the " \
          "object's instance variables to set values."
study.submission_date = str(datetime.datetime.today())
study.public_release_date = str(datetime.datetime.today())
study.sources = [Source(name="source1")]
study.samples = [Sample(name="sample1")]
study.protocols = [Protocol(name="sample collection"),
                   Protocol(name="data analysis with Galaxy",
                            uri="https://doi.org/10.5464/workflow.cwl",
                            protocol_type=OntologyAnnotation(term="data transformation"),
                            parameters=[ProtocolParameter(parameter_name=OntologyAnnotation(term="genome assembly")),
                                        ProtocolParameter(parameter_name=OntologyAnnotation(term="cut-off value"))]),
                    Protocol(name="data visualization with Intermine",
                             uri="https://intermine.org/10.5464/network.svg",
                             protocol_type=OntologyAnnotation(term="data visualization"),
                                                                )]
study.process_sequence = [Process(executes_protocol=study.protocols[-1], inputs=[study.sources[-1]], outputs=[study.samples[-1]])]
investigation.studies = [study]



In [3]:
# Let's see the object :
investigation

isatools.model.Investigation(identifier='', filename='', title='', submission_date='', public_release_date='', ontology_source_references=[], publications=[], contacts=[], studies=[isatools.model.Study(filename='s_study.txt', identifier='S1', title='My Simple ISA Study', description='We could alternatively use the class constructor's parameters to set some default values at the time of creation, however we want to demonstrate how to use the object's instance variables to set values.', submission_date='2021-01-29 22:44:51.219886', public_release_date='2021-01-29 22:44:51.219945', contacts=[], design_descriptors=[], publications=[], factors=[], protocols=[isatools.model.Protocol(name='sample collection', protocol_type=isatools.model.OntologyAnnotation(term='', term_source=None, term_accession='', comments=[]), uri='', version='', parameters=[], components=[], comments=[]), isatools.model.Protocol(name='data analysis with Galaxy', protocol_type=isatools.model.OntologyAnnotation(term='data

## Writing to ISA-Tab

In [4]:
from isatools.isatab import dumps
print(dumps(investigation))

/var/folders/5n/rl6lqnks4rqb59pbtpvvntqw0000gr/T/tmpaoi99swh/i_investigation.txt
ONTOLOGY SOURCE REFERENCE
Term Source Name
Term Source File
Term Source Version
Term Source Description
INVESTIGATION
Investigation Identifier	
Investigation Title	
Investigation Description	
Investigation Submission Date	
Investigation Public Release Date	
INVESTIGATION PUBLICATIONS
Investigation PubMed ID
Investigation Publication DOI
Investigation Publication Author List
Investigation Publication Title
Investigation Publication Status
Investigation Publication Status Term Accession Number
Investigation Publication Status Term Source REF
INVESTIGATION CONTACTS
Investigation Person Last Name
Investigation Person First Name
Investigation Person Mid Initials
Investigation Person Email
Investigation Person Phone
Investigation Person Fax
Investigation Person Address
Investigation Person Affiliation
Investigation Person Roles
Investigation Person Roles Term Accession Number
Investigation Person Roles Term Sour

## Writing to ISA-JSON

In [5]:
import json
from isatools.isajson import ISAJSONEncoder
print(json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "comments": [],
    "description": "",
    "identifier": "",
    "ontologySourceReferences": [],
    "people": [],
    "publicReleaseDate": "",
    "publications": [],
    "studies": [
        {
            "assays": [],
            "characteristicCategories": [],
            "comments": [],
            "description": "We could alternatively use the class constructor's parameters to set some default values at the time of creation, however we want to demonstrate how to use the object's instance variables to set values.",
            "factors": [],
            "filename": "s_study.txt",
            "identifier": "S1",
            "materials": {
                "otherMaterials": [],
                "samples": [
                    {
                        "@id": "#sample/4443726224",
                        "characteristics": [],
                        "factorValues": [],
                        "name": "sample1"
                    }
                ],
                "sources": 